In [1]:
import lightgbm as lgb
from sklearn.grid_search import GridSearchCV
import time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
import pandas as pd
data_tree_train = pd.read_csv('data_tree_train_final.csv',encoding='gbk')
data_tree_test = pd.read_csv('data_tree_test_final.csv',encoding='gbk')

# data_tree_train = data_tree_train.fillna(0)
# data_tree_test = data_tree_test.fillna(0)

data_tree_train_1 = data_tree_train[data_tree_train['label']==1]
data_tree_train_0 = data_tree_train[data_tree_train['label']==0]

train0,test0 = train_test_split(data_tree_train_0, test_size=0.15)
train1,test1 = train_test_split(data_tree_train_1, test_size=0.15)

test = pd.concat([test0,test1],ignore_index=True)

# gbdtTrain0, otherTrain0 = train_test_split(train0, test_size=0.5)
# gbdtTrain1, otherTrain1 = train_test_split(train1, test_size=0.5)

# gbdtTrain1 = gbdtTrain1.sample(6000,replace=True)
# otherTrain1 = otherTrain1.sample(6000,replace=True)

# gbdtTrain = pd.concat([gbdtTrain1,gbdtTrain0],ignore_index=True)
# otherTrain = pd.concat([otherTrain1,otherTrain0],ignore_index=True)

In [3]:
train1Plus = train1.sample(13000,replace=True)
All = pd.concat([train0,train1Plus],ignore_index=True)

In [4]:
All.label.value_counts()

0    16150
1    13000
Name: label, dtype: int64

In [5]:
train1Plus = data_tree_train_1.sample(14000,replace=True)
train = pd.concat([data_tree_train_0,train1Plus],ignore_index=True)
train.label.value_counts()

0    19000
1    14000
Name: label, dtype: int64

In [6]:
pd.set_option('display.max_colwidth',100)
pd.set_option('display.max_columns', 1000)
target='label'
IDcol = '个人编码'
predictors = [x for x in data_tree_train.columns if x not in [target, IDcol]]

In [7]:
lgb111 = lgb.LGBMClassifier(
        boosting_type='goss', num_leaves=32, max_depth=-1, 
        learning_rate=0.1, n_estimators=256, max_bin=255, subsample_for_bin=50000, 
        min_split_gain=0, min_child_weight=1, min_child_samples=10, 
        subsample=1, subsample_freq=1, colsample_bytree=1, reg_alpha=0, reg_lambda=0, 
        seed=0, nthread=-1, silent=True)
lgb111.fit(train[predictors], train['label'])

LGBMClassifier(boosting_type='goss', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=1,
        min_split_gain=0, n_estimators=256, nthread=-1, num_leaves=32,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [10]:
feaImp = pd.Series(lgb111.booster().feature_importance()).sort_values(ascending=False)#获得重要性不为0的特征
goodFeature = feaImp[feaImp>30].index
features = data_tree_train_0[predictors].columns[goodFeature]
features

D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function booster is deprecated; Use attribute booster_ instead.
  warnings.warn(msg, category=DeprecationWarning)


Index(['单个顺序号数量_sum', '当月就诊次数var', '本次审批金额_sum', '当月就诊次数max', '操作时间_月_var',
       '当天就诊次数mean', '基本医疗保险统筹基金支付金额_sum', '当天就诊次数var', '药品费发生金额_sum',
       '三目统计项目_1.0_sum', '单个顺序号数量_mean', '操作时间_月_9.0_mean', '单个医院就诊最多次数/顺序号数量',
       '操作时间_月_8.0_mean', '三目统计项目_3.0_mean', '操作时间_月_12.0_mean',
       '操作时间_dayofweek_var', '中成药费发生金额_max', '操作时间_月_mean', '非账户支付金额_max',
       '操作时间_dayofweek_2.0_mean', '单个顺序号数量_var', '中成药费发生金额_sum',
       '医用材料发生金额_sum', '单价_min_mean', '当月就诊次数mean', '数量_sum_sum',
       '操作时间_月_10.0_mean', '操作时间_dayofweek_5.0_mean', '操作时间_dayofweek_mean',
       '基本医疗保险统筹基金支付金额_max', '起付标准以上自负比例金额_sum', '医用材料发生金额_var', '数量_mean_sum',
       '单价_mean_max', '非账户支付金额_sum', '中成药费发生金额_mean', '单个医院就诊最多次数',
       '操作时间_月_7.0_mean', '三目统计项目_1.0_mean', '药品费申报金额_max', '操作时间_月_11.0_mean',
       '单个医院就诊最少次数/顺序号数量', '平均医院数量', '三目统计项目_3.0_var', '单价_sum_max',
       '基本医疗保险个人账户支付金额_max', '数量_sum_mean', '可用账户报销金额_sum', '单价_sum_sum',
       '药品费发生金额_max', '单价_min_max', '单个医院就诊最少次数', '治疗费

In [11]:
feaImp

431    125
555    112
354     99
552     93
466     84
550     84
350     82
551     77
324     70
415     69
107     68
46      68
546     67
45      66
93      63
49      60
467     60
218     60
34      58
245     58
52      57
539     57
326     54
341     54
100     53
554     52
430     52
47      51
55      50
35      50
      ... 
643      0
642      0
641      0
680      0
682      0
725      0
714      0
724      0
723      0
722      0
721      0
720      0
719      0
718      0
717      0
716      0
715      0
713      0
683      0
712      0
711      0
710      0
709      0
708      0
707      0
706      0
694      0
690      0
685      0
508      0
Length: 1018, dtype: int32

In [15]:
data_tree_train_0[predictors].columns[431]

'单个顺序号数量_sum'

In [9]:
start = time.time()
param_test1 = {
#     'num_leaves':np.array([10,32,100]),
#     'max_depth':np.array([5,10,-1]),
#     'min_child_weight':np.array([1,3,5,10]),
#     'gamma':[i/10.0 for i in range(0,5)],
    'subsample_for_bin':np.array([100,500,50000])
}
gsearch1 = GridSearchCV(estimator = lgb.LGBMClassifier( boosting_type='gbdt', num_leaves=10, max_depth=7, 
                                                       learning_rate=0.1, n_estimators=256, max_bin=255, subsample_for_bin=50000, 
                                                       min_split_gain=0, min_child_weight=1, min_child_samples=10, 
                                                       subsample=1, subsample_freq=1, colsample_bytree=1, reg_alpha=0, reg_lambda=0, 
                                                       seed=0, nthread=-1, silent=True), 
                       param_grid = param_test1, scoring='f1',n_jobs=4,iid=False, cv=8)
gsearch1.fit(data_tree_train[features],data_tree_train[target])
end = time.time()
print('运行时间：')
(end-start)/60

运行时间：


1.4884151697158814

In [10]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.47766, std: 0.05208, params: {'subsample_for_bin': 100},
  mean: 0.47766, std: 0.05208, params: {'subsample_for_bin': 500},
  mean: 0.47766, std: 0.05208, params: {'subsample_for_bin': 50000}],
 {'subsample_for_bin': 100},
 0.4776564234768295)

In [183]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.45554, std: 0.05803, params: {'max_depth': 5, 'n_estimators': 100, 'num_leaves': 10},
  mean: 0.45588, std: 0.05666, params: {'max_depth': 5, 'n_estimators': 100, 'num_leaves': 32},
  mean: 0.45588, std: 0.05666, params: {'max_depth': 5, 'n_estimators': 100, 'num_leaves': 100},
  mean: 0.48016, std: 0.06289, params: {'max_depth': 5, 'n_estimators': 500, 'num_leaves': 10},
  mean: 0.47082, std: 0.05794, params: {'max_depth': 5, 'n_estimators': 500, 'num_leaves': 32},
  mean: 0.47082, std: 0.05794, params: {'max_depth': 5, 'n_estimators': 500, 'num_leaves': 100},
  mean: 0.46980, std: 0.05838, params: {'max_depth': 5, 'n_estimators': 1000, 'num_leaves': 10},
  mean: 0.47427, std: 0.05347, params: {'max_depth': 5, 'n_estimators': 1000, 'num_leaves': 32},
  mean: 0.47427, std: 0.05347, params: {'max_depth': 5, 'n_estimators': 1000, 'num_leaves': 100},
  mean: 0.46271, std: 0.06009, params: {'max_depth': 10, 'n_estimators': 100, 'num_leaves': 10},
  mean: 0.46635, std: 0.05774, pa

In [136]:
lgb0 = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=32, max_depth=-1, 
        learning_rate=0.1, n_estimators=1024, max_bin=255, subsample_for_bin=50000, 
        min_split_gain=0, min_child_weight=1, min_child_samples=10, 
        subsample=1, subsample_freq=1, colsample_bytree=1, reg_alpha=0, reg_lambda=0, 
        seed=0, nthread=-1, silent=True)
lgb0.fit(train[features], train['label'])

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=1,
        min_split_gain=0, n_estimators=1024, nthread=-1, num_leaves=32,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [137]:
lgb1 = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=16, max_depth=7, 
        learning_rate=0.05, n_estimators=256, max_bin=255, subsample_for_bin=50000, 
        min_split_gain=0, min_child_weight=1, min_child_samples=10, 
        subsample=1, subsample_freq=1, colsample_bytree=1, reg_alpha=0, reg_lambda=0, 
        seed=0, nthread=-1, silent=True)
lgb1.fit(train[features], train['label'])

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.05, max_bin=255, max_depth=7,
        max_drop=50, min_child_samples=10, min_child_weight=1,
        min_split_gain=0, n_estimators=256, nthread=-1, num_leaves=16,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [138]:
lgb2 = lgb.LGBMClassifier(
        boosting_type='dart', num_leaves=32, max_depth=10, 
        learning_rate=0.05, n_estimators=2048, max_bin=255, subsample_for_bin=50000, 
        min_split_gain=0, min_child_weight=1, min_child_samples=10, 
        subsample=1, subsample_freq=1, colsample_bytree=1, reg_alpha=0, reg_lambda=0, 
        seed=0, nthread=-1, silent=True)
lgb2.fit(train[features], train['label'])

LGBMClassifier(boosting_type='dart', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.05, max_bin=255, max_depth=10,
        max_drop=50, min_child_samples=10, min_child_weight=1,
        min_split_gain=0, n_estimators=2048, nthread=-1, num_leaves=32,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [139]:
lgb3 = lgb.LGBMClassifier(
        boosting_type='dart', num_leaves=100, max_depth=15, 
        learning_rate=0.1, n_estimators=128, max_bin=255, subsample_for_bin=50000, 
        min_split_gain=0, min_child_weight=1, min_child_samples=10, 
        subsample=1, subsample_freq=1, colsample_bytree=1, reg_alpha=0, reg_lambda=0, 
        seed=0, nthread=-1, silent=True)
lgb3.fit(train[features], train['label'])

LGBMClassifier(boosting_type='dart', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=15,
        max_drop=50, min_child_samples=10, min_child_weight=1,
        min_split_gain=0, n_estimators=128, nthread=-1, num_leaves=100,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [140]:
lgb4 = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=100, max_depth=30, 
        learning_rate=0.1, n_estimators=500, max_bin=255, subsample_for_bin=50000, 
        min_split_gain=0, min_child_weight=1, min_child_samples=10, 
        subsample=1, subsample_freq=1, colsample_bytree=1, reg_alpha=0, reg_lambda=0, 
        seed=0, nthread=-1, silent=True)
lgb4.fit(train[features], train['label'])

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=30,
        max_drop=50, min_child_samples=10, min_child_weight=1,
        min_split_gain=0, n_estimators=500, nthread=-1, num_leaves=100,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [120]:
lgb5 = lgb.LGBMClassifier(
        boosting_type='goss', num_leaves=28, max_depth=20, 
        learning_rate=0.1, n_estimators=700, max_bin=255, subsample_for_bin=50000, 
        min_split_gain=0, min_child_weight=1, min_child_samples=10, 
        subsample=1, subsample_freq=1, colsample_bytree=1, reg_alpha=0, reg_lambda=0, 
        seed=0, nthread=-1, silent=True)
lgb5.fit(All[features], All['label'])

LGBMClassifier(boosting_type='goss', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=20,
        max_drop=50, min_child_samples=10, min_child_weight=1,
        min_split_gain=0, n_estimators=700, nthread=-1, num_leaves=28,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [121]:
lgb6 = lgb.LGBMClassifier(
        boosting_type='goss', num_leaves=30, max_depth=32, 
        learning_rate=0.1, n_estimators=256, max_bin=255, subsample_for_bin=50000, 
        min_split_gain=0, min_child_weight=1, min_child_samples=10, 
        subsample=1, subsample_freq=1, colsample_bytree=1, reg_alpha=0, reg_lambda=0, 
        seed=0, nthread=-1, silent=True)
lgb6.fit(All[features], All['label'])

LGBMClassifier(boosting_type='goss', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=32,
        max_drop=50, min_child_samples=10, min_child_weight=1,
        min_split_gain=0, n_estimators=256, nthread=-1, num_leaves=30,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [129]:
# lgbFeatures0 = pd.DataFrame(lgb0.apply(All[features]))
# data_lr_trainPlus00 = pd.concat([All[features],lgbFeatures0],axis=1)
# data_lr_trainPlus00

,当月就诊次数var,单个顺序号数量_sum,当天就诊次数mean,本次审批金额_sum,单个医院就诊最多次数/顺序号数量,当月就诊次数max,当天就诊次数var,基本医疗保险统筹基金支付金额_sum,三目统计项目_1.0_sum,操作时间_月_var,三目统计项目_3.0_mean,操作时间_dayofweek_var,中成药费发生金额_max,药品费发生金额_sum,起付标准以上自负比例金额_sum,三目统计项目_3.0_var,单个顺序号数量_mean,操作时间_月_9.0_var,单个顺序号数量_var,医用材料发生金额_sum,药品费申报金额_max,操作时间_月_12.0_mean,非账户支付金额_max,非账户支付金额_sum,三目统计项目_1.0_var,操作时间_月_9.0_mean,中成药费发生金额_sum,单个医院就诊最少次数/顺序号数量,操作时间_月_mean,操作时间_月_11.0_var,基本医疗保险统筹基金支付金额_max,单价_min_mean,操作时间_月_11.0_mean,操作时间_月_8.0_mean,操作时间_月_8.0_var,单个医院就诊最多次数,起付标准以上自负比例金额_max,交易时间_月_var,医用材料发生金额_var,操作时间_dayofweek_mean,数量_sum_sum,药品费申报金额_sum,单价_mean_max,操作时间_月_10.0_mean,非账户支付金额_mean,三目统计项目_1.0_mean,数量_mean_sum,中成药费发生金额_mean,起付标准以上自负比例金额_mean,数量_sum_mean,数量_min_sum,基本医疗保险个人账户支付金额_max,当月就诊次数mean,可用账户报销金额_mean,单价_max_mean,单价_sum_max,单价_mean_mean,单价_sum_sum,药品费发生金额_max,基本医疗保险个人账户支付金额_var,三目统计项目_3.0_sum,当月就诊次数min,单价_min_var,药品费自费金额_max,单个医院就诊最少次数,药品费自费金额_sum,交易时间_月_mean,基本医疗保险统筹基金支付金额_var,交易时间_dayofweek_var,基本医疗保险个人账户支付金额_mean,数量_min_var,可用账户报销金额_sum,单价_max_var,单价_sum_var,操作时间_dayofweek_1.0_var,单价_min_max,三目统计项目_6.0_sum,非账户支付金额_var,操作时间_dayofweek_2.0_mean,操作时间_月_7.0_mean,单价_sum_mean,医院编码_104.0,操作时间_月_10.0_var,本次审批金额_mean,中草药费发生金额_max,基本医疗保险个人账户支付金额_sum,单个顺序号数量_max,单价_min_sum,操作时间_月_12.0_var,数量_mean_max,基本医疗保险统筹基金支付金额_mean,治疗费发生金额_mean,医用材料发生金额_max,操作时间_dayofweek_4.0_mean,操作时间_dayofweek_2.0_var,数量_max_sum,操作时间_月_7.0_var,药品费自费金额_mean,药品费自费金额_var,操作时间_月_12.0_sum,操作时间_dayofweek_5.0_mean,医用材料发生金额_mean,数量_sum_max,治疗费申报金额_var,中成药费发生金额_var,数量_min_mean,三目统计项目_2.0_sum,可用账户报销金额_max,本次审批金额_max,数量_max_var,药品费发生金额_mean,药品费申报金额_mean,药品费发生金额_var,操作时间_dayofweek_6.0_sum,操作时间_dayofweek_0.0_sum,治疗费发生金额_var,起付标准以上自负比例金额_var,顺序号数量,数量_mean_var,操作时间_dayofweek_1.0_mean,治疗费发生金额_max,单价_mean_var,操作时间_dayofweek_3.0_mean,治疗费发生金额_sum,贵重药品发生金额_var,操作时间_dayofweek_4.0_sum,平均医院数量,数量_max_max,三目统计项目_6.0_mean,可用账户报销金额_var,医院编码_188.0,单价_mean_sum,贵重药品发生金额_sum,操作时间_月_7.0_sum,交易时间_月_7.0_mean,交易时间_月_10.0_var,三目统计项目_7.0_sum,三目统计项目_2.0_mean,三目统计项目_2.0_var,操作时间_dayofweek_3.0_var,交易时间_月_7.0_var,数量_min_max,药品费申报金额_var,单价_sum_min,操作时间_dayofweek_0.0_mean,数量_sum_var,操作时间_月_8.0_sum,贵重药品发生金额_mean,操作时间_dayofweek_2.0_sum,检查费发生金额_mean,操作时间_dayofweek_3.0_sum,数量_mean_mean,操作时间_月_9.0_sum,操作时间_dayofweek_6.0_mean,检查费发生金额_sum,操作时间_dayofweek_sum,操作时间_dayofweek_6.0_var,三目统计项目_7.0_mean,操作时间_dayofweek_5.0_var,操作时间_dayofweek_4.0_var,交易时间_月_10.0_mean,单价_mean_min,检查费申报金额_sum,三目统计项目_6.0_var,当天就诊次数max,三目统计项目_1.0_max,本次审批金额_var,基本医疗保险统筹基金支付金额_min,数量_max_mean,操作时间_月_11.0_sum,其它发生金额_sum,单价_max_min,单价_max_max,治疗费申报金额_max,交易时间_dayofweek_mean,三目统计项目_7.0_var,操作时间_dayofweek_5.0_sum,非账户支付金额_min,医院编码_183.0,贵重药品发生金额_max,交易时间_dayofweek_6.0_var,检查费发生金额_max,其它发生金额_var,操作时间_月_sum,医用材料费自费金额_sum,治疗费申报金额_mean,操作时间_dayofweek_0.0_var,检查费发生金额_var,交易时间_dayofweek_2.0_var,操作时间_dayofweek_1.0_sum,可用账户报销金额_min,单价_max_sum,交易时间_月_10.0_sum,医院编码_197.0,交易时间_dayofweek_0.0_sum,治疗费申报金额_sum,医院编码_125.0,检查费申报金额_mean,三目统计项目_2.0_max,医院编码_194.0,医院编码_180.0,中草药费发生金额_sum,单个医院就诊平均次数,医疗救助医院申请_max,单价_min_min,当天就诊次数min,其它发生金额_mean,交易时间_dayofweek_1.0_var,起付标准以上自负比例金额_min,中草药费发生金额_var,交易时间_dayofweek_3.0_var,交易时间_dayofweek_0.0_var,检查费申报金额_max,医院编码_5.0,操作时间_月_10.0_sum,三目统计项目_6.0_max,医院编码_587.0,检查费申报金额_var,中草药费发生金额_mean,治疗费自费金额_sum,医院编码_558.0,医院编码_110.0,医院编码_186.0,交易时间_dayofweek_0.0_mean,医院编码_143.0,医院编码_160.0,医院编码_150.0,数量_mean_min,医院编码_1079.0,医院编码_140.0,医用材料费自费金额_mean,民政救助补助金额_mean,交易时间_dayofweek_2.0_sum,交易时间_dayofweek_3.0_mean,医院编码_193.0,医院数量,交易时间_dayofweek_4.0_var,医院编码_196.0,交易时间_dayofweek_5.0_mean,交易时间_dayofweek_2.0_mean,治疗费自费金额_mean,医院编码_145.0,民政救助补助金额_min,医用材料费自费金额_var,其它发生金额_max,医院编码_754.0,医院编码_170.0,交易时间_dayofweek_5.0_var,医院编码_567.0,医院编码_444.0,交易时间_月_7.0_sum,交易时间_dayofweek_sum,医院编码_1254.0,民政救助补助金额_max,医院编码_178.0,医院编码_7.0,医院编码_98.0,医院编码_896.0,医院编码_32.0,床位费发生金额_mean,贵重检查费金额_sum,医院编码_107.0,民政救助补助金额_sum,治疗费自费金额_var,三目统计项目_5.0_var,医院编码_620.0,医用材料费自费金额_max,医院编码_30.0,本次审批金额_min,医院编码_136.0,医院编码_122.0,医院编码_189.0,贵重检查费金额_var,医院编码_626.0,医院编码_26.0,医院编码_69.0,医院编码_65.0,医院编码_716.0,治疗费自费金额_max,贵重检查费金额_max,医院编码_636.0,医院编码_

In [161]:
# lgb1 = lgb.LGBMClassifier(
#         boosting_type='gbdt', num_leaves=100, max_depth=-1, 
#         learning_rate=0.1, n_estimators=128, max_bin=255, subsample_for_bin=50000, 
#         min_split_gain=0, min_child_weight=1, min_child_samples=10, 
#         subsample=1, subsample_freq=1, colsample_bytree=1, reg_alpha=10, reg_lambda=0, 
#         seed=0, nthread=-1, silent=True)
# lgb1.fit(lgbFeatures0, All['label'])

D:\Anaconda3\lib\site-packages\lightgbm-0.1-py3.6.egg\lightgbm\engine.py:163: UserWarning: Using Pandas (default) integer column names, not column indexes. You can use indexes with DataFrame.values.
  booster = Booster(params=params, train_set=train_set)


LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=1,
        min_split_gain=0, n_estimators=128, nthread=-1, num_leaves=100,
        objective='binary', reg_alpha=10, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [131]:
# testFeatures = pd.DataFrame(lgb0.apply(test[features]))
# data_lr_test0 = pd.concat([test[features],testFeatures],axis=1)
# data_lr_test0

,当月就诊次数var,单个顺序号数量_sum,当天就诊次数mean,本次审批金额_sum,单个医院就诊最多次数/顺序号数量,当月就诊次数max,当天就诊次数var,基本医疗保险统筹基金支付金额_sum,三目统计项目_1.0_sum,操作时间_月_var,三目统计项目_3.0_mean,操作时间_dayofweek_var,中成药费发生金额_max,药品费发生金额_sum,起付标准以上自负比例金额_sum,三目统计项目_3.0_var,单个顺序号数量_mean,操作时间_月_9.0_var,单个顺序号数量_var,医用材料发生金额_sum,药品费申报金额_max,操作时间_月_12.0_mean,非账户支付金额_max,非账户支付金额_sum,三目统计项目_1.0_var,操作时间_月_9.0_mean,中成药费发生金额_sum,单个医院就诊最少次数/顺序号数量,操作时间_月_mean,操作时间_月_11.0_var,基本医疗保险统筹基金支付金额_max,单价_min_mean,操作时间_月_11.0_mean,操作时间_月_8.0_mean,操作时间_月_8.0_var,单个医院就诊最多次数,起付标准以上自负比例金额_max,交易时间_月_var,医用材料发生金额_var,操作时间_dayofweek_mean,数量_sum_sum,药品费申报金额_sum,单价_mean_max,操作时间_月_10.0_mean,非账户支付金额_mean,三目统计项目_1.0_mean,数量_mean_sum,中成药费发生金额_mean,起付标准以上自负比例金额_mean,数量_sum_mean,数量_min_sum,基本医疗保险个人账户支付金额_max,当月就诊次数mean,可用账户报销金额_mean,单价_max_mean,单价_sum_max,单价_mean_mean,单价_sum_sum,药品费发生金额_max,基本医疗保险个人账户支付金额_var,三目统计项目_3.0_sum,当月就诊次数min,单价_min_var,药品费自费金额_max,单个医院就诊最少次数,药品费自费金额_sum,交易时间_月_mean,基本医疗保险统筹基金支付金额_var,交易时间_dayofweek_var,基本医疗保险个人账户支付金额_mean,数量_min_var,可用账户报销金额_sum,单价_max_var,单价_sum_var,操作时间_dayofweek_1.0_var,单价_min_max,三目统计项目_6.0_sum,非账户支付金额_var,操作时间_dayofweek_2.0_mean,操作时间_月_7.0_mean,单价_sum_mean,医院编码_104.0,操作时间_月_10.0_var,本次审批金额_mean,中草药费发生金额_max,基本医疗保险个人账户支付金额_sum,单个顺序号数量_max,单价_min_sum,操作时间_月_12.0_var,数量_mean_max,基本医疗保险统筹基金支付金额_mean,治疗费发生金额_mean,医用材料发生金额_max,操作时间_dayofweek_4.0_mean,操作时间_dayofweek_2.0_var,数量_max_sum,操作时间_月_7.0_var,药品费自费金额_mean,药品费自费金额_var,操作时间_月_12.0_sum,操作时间_dayofweek_5.0_mean,医用材料发生金额_mean,数量_sum_max,治疗费申报金额_var,中成药费发生金额_var,数量_min_mean,三目统计项目_2.0_sum,可用账户报销金额_max,本次审批金额_max,数量_max_var,药品费发生金额_mean,药品费申报金额_mean,药品费发生金额_var,操作时间_dayofweek_6.0_sum,操作时间_dayofweek_0.0_sum,治疗费发生金额_var,起付标准以上自负比例金额_var,顺序号数量,数量_mean_var,操作时间_dayofweek_1.0_mean,治疗费发生金额_max,单价_mean_var,操作时间_dayofweek_3.0_mean,治疗费发生金额_sum,贵重药品发生金额_var,操作时间_dayofweek_4.0_sum,平均医院数量,数量_max_max,三目统计项目_6.0_mean,可用账户报销金额_var,医院编码_188.0,单价_mean_sum,贵重药品发生金额_sum,操作时间_月_7.0_sum,交易时间_月_7.0_mean,交易时间_月_10.0_var,三目统计项目_7.0_sum,三目统计项目_2.0_mean,三目统计项目_2.0_var,操作时间_dayofweek_3.0_var,交易时间_月_7.0_var,数量_min_max,药品费申报金额_var,单价_sum_min,操作时间_dayofweek_0.0_mean,数量_sum_var,操作时间_月_8.0_sum,贵重药品发生金额_mean,操作时间_dayofweek_2.0_sum,检查费发生金额_mean,操作时间_dayofweek_3.0_sum,数量_mean_mean,操作时间_月_9.0_sum,操作时间_dayofweek_6.0_mean,检查费发生金额_sum,操作时间_dayofweek_sum,操作时间_dayofweek_6.0_var,三目统计项目_7.0_mean,操作时间_dayofweek_5.0_var,操作时间_dayofweek_4.0_var,交易时间_月_10.0_mean,单价_mean_min,检查费申报金额_sum,三目统计项目_6.0_var,当天就诊次数max,三目统计项目_1.0_max,本次审批金额_var,基本医疗保险统筹基金支付金额_min,数量_max_mean,操作时间_月_11.0_sum,其它发生金额_sum,单价_max_min,单价_max_max,治疗费申报金额_max,交易时间_dayofweek_mean,三目统计项目_7.0_var,操作时间_dayofweek_5.0_sum,非账户支付金额_min,医院编码_183.0,贵重药品发生金额_max,交易时间_dayofweek_6.0_var,检查费发生金额_max,其它发生金额_var,操作时间_月_sum,医用材料费自费金额_sum,治疗费申报金额_mean,操作时间_dayofweek_0.0_var,检查费发生金额_var,交易时间_dayofweek_2.0_var,操作时间_dayofweek_1.0_sum,可用账户报销金额_min,单价_max_sum,交易时间_月_10.0_sum,医院编码_197.0,交易时间_dayofweek_0.0_sum,治疗费申报金额_sum,医院编码_125.0,检查费申报金额_mean,三目统计项目_2.0_max,医院编码_194.0,医院编码_180.0,中草药费发生金额_sum,单个医院就诊平均次数,医疗救助医院申请_max,单价_min_min,当天就诊次数min,其它发生金额_mean,交易时间_dayofweek_1.0_var,起付标准以上自负比例金额_min,中草药费发生金额_var,交易时间_dayofweek_3.0_var,交易时间_dayofweek_0.0_var,检查费申报金额_max,医院编码_5.0,操作时间_月_10.0_sum,三目统计项目_6.0_max,医院编码_587.0,检查费申报金额_var,中草药费发生金额_mean,治疗费自费金额_sum,医院编码_558.0,医院编码_110.0,医院编码_186.0,交易时间_dayofweek_0.0_mean,医院编码_143.0,医院编码_160.0,医院编码_150.0,数量_mean_min,医院编码_1079.0,医院编码_140.0,医用材料费自费金额_mean,民政救助补助金额_mean,交易时间_dayofweek_2.0_sum,交易时间_dayofweek_3.0_mean,医院编码_193.0,医院数量,交易时间_dayofweek_4.0_var,医院编码_196.0,交易时间_dayofweek_5.0_mean,交易时间_dayofweek_2.0_mean,治疗费自费金额_mean,医院编码_145.0,民政救助补助金额_min,医用材料费自费金额_var,其它发生金额_max,医院编码_754.0,医院编码_170.0,交易时间_dayofweek_5.0_var,医院编码_567.0,医院编码_444.0,交易时间_月_7.0_sum,交易时间_dayofweek_sum,医院编码_1254.0,民政救助补助金额_max,医院编码_178.0,医院编码_7.0,医院编码_98.0,医院编码_896.0,医院编码_32.0,床位费发生金额_mean,贵重检查费金额_sum,医院编码_107.0,民政救助补助金额_sum,治疗费自费金额_var,三目统计项目_5.0_var,医院编码_620.0,医用材料费自费金额_max,医院编码_30.0,本次审批金额_min,医院编码_136.0,医院编码_122.0,医院编码_189.0,贵重检查费金额_var,医院编码_626.0,医院编码_26.0,医院编码_69.0,医院编码_65.0,医院编码_716.0,治疗费自费金额_max,贵重检查费金额_max,医院编码_636.0,医院编码_

In [15]:
from sklearn.metrics import f1_score

In [141]:
testPre0 = lgb0.predict(data_tree_test[features])
testPre1 = lgb1.predict(data_tree_test[features])
testPre2 = lgb2.predict(data_tree_test[features])
testPre3 = lgb3.predict(data_tree_test[features])
testPre4 = lgb4.predict(data_tree_test[features])
# testPre5 = lgb5.predict(test[features])
# testPre6 = lgb6.predict(test[features])

In [142]:
res1 = testPre0+testPre1+testPre2+testPre3+testPre4
res1[res1<3] = 0
res1[res1>2] = 1
res1

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [143]:
data_tree_test['lrpre'] = res1
data_tree_test['lrpre'].value_counts()

0    3833
1     167
Name: lrpre, dtype: int64

In [144]:
lrRes = data_tree_test[['个人编码','lrpre']]
lrRes.to_csv('ronghe5.csv',index=None,header=None)

In [135]:
f1_score(testPre0, test['label'])

0.44444444444444442

In [139]:
testPre1 = lgb1.predict(testFeatures)
sum(testPre1==1)

D:\Anaconda3\lib\site-packages\lightgbm-0.1-py3.6.egg\lightgbm\sklearn.py:598: UserWarning: Using Pandas (default) integer column names, not column indexes. You can use indexes with DataFrame.values.
  class_probs = self.predict_proba(X, raw_score, num_iteration)


58

In [88]:
f1_score(testPre0, test['label'])

0.5423728813559322

In [26]:
lgbFeaturesTest = pd.DataFrame(lgb0.apply(data_tree_test[features]))
data_lr_test = pd.concat([data_tree_test[features],lgbFeaturesTest],axis=1)

In [30]:
ggP = lgb1.predict(data_lr_test)
data_tree_test['ggP'] = ggP
data_tree_test['ggP'].value_counts()

0    3868
1     132
Name: ggP, dtype: int64

In [182]:
Pre = lgb0.predict(data_tree_test[features])
data_tree_test['lrpre'] = Pre
data_tree_test['lrpre'].value_counts()

0    3791
1     209
Name: lrpre, dtype: int64

In [31]:
lrRes = data_tree_test[['个人编码','ggP']]
lrRes.to_csv('gbdt+dart_L1_10.csv',index=None,header=None)